## Classification

The purpose of this notebook is to start modeling the data! As tiresome as it can be, tidying and exploring our data is a vital component to any data analysis. If we had jumped straight to the modeling step, we would have created a faulty classification model.



## 1. Reading Data

> This section is dedicated to reading the cleaned dataset and importing all libraries.

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

mutation_data_clean = pd.read_csv("data/mutation_data_clean.csv")